In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking, Dropout, InputLayer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
symptoms = ["зудящий раздражение_кожи извержения_кожи дихроичные_пятна", "дрожащий озноб стекающий_с_глаз", "боль_в_желудке повышенная_кислотность рвота кашель боль_в_груди."]
disease = ["Грибковая инфекция", "Аллергия", "ГЕРД"]

In [9]:
max_words_count=100
tokenizer=Tokenizer(num_words=max_words_count)
tokenizer.fit_on_texts(symptoms)
data_x = tokenizer.texts_to_sequences(symptoms)

In [10]:
data_x

[[4, 5, 1, 6, 1, 7, 8],
 [9, 10, 11, 12, 13],
 [2, 3, 14, 15, 16, 17, 18, 2, 3, 19]]

In [13]:
max_len = max([len(x) for x in data_x])
data_x = pad_sequences(data_x, maxlen=max_len)

In [14]:
data_x

array([[ 0,  0,  0,  4,  5,  1,  6,  1,  7,  8],
       [ 0,  0,  0,  0,  0,  9, 10, 11, 12, 13],
       [ 2,  3, 14, 15, 16, 17, 18,  2,  3, 19]], dtype=int32)

In [15]:
# Кодирование меток заболеваний
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(disease)

In [16]:
data_y

array([2, 0, 1])

In [17]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [19]:
# Создание модели LSTM
model = Sequential()
model.add(InputLayer(shape=(max_len,)))
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=1, batch_size=2, validation_data=(X_test, y_test))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.1041 - val_accuracy: 0.0000e+00 - val_loss: 1.1076


In [20]:
# Тестовая классификация
# На сайте будет подобное иcпользование, где вместо test_symp будет подоваться множество симптомов юзера
test_symp = 'зудящий раздражение_кожи извержения_кожи дихроичные_пятна'
t = tokenizer.texts_to_sequences([test_symp])
t_pad = pad_sequences(t, maxlen = max_len)

In [21]:
#Получение результатов классификации 
res = model.predict(t_pad)
label_res = {
    "Disease": label_encoder.classes_,
    "values": res[0]
}
res_frame = pd.DataFrame(label_res)
res_frame.sort_values('values', ascending=False)
#Таблица с результатами классификации в порядке убывания вероятности

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


,Disease,values
0,Аллергия,0.336029
1,ГЕРД,0.333631
2,Грибковая инфекция,0.330340
